# Defining Functions

In [ ]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
# from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.losses import binary_crossentropy
import os

In [ ]:
def data_generator(dataset, batch_size , epochs=1):
  num_datapoints = len(dataset)
  for e in range(epochs):
    for offset in range(0, num_datapoints, batch_size):
      X_batch = []
      y_batch = []
      batch_dataset = dataset[offset:offset + batch_size]
      shuffled_batch = batch_dataset.sample(frac=1)
      # print(shuffled_batch)

      path_list = shuffled_batch['Img_path']
      labels_list = shuffled_batch['Fracture_Labels']

      for img_path in path_list:
        img = cv2.imread(img_path)
        # gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        X_batch.append(img)

      for x in labels_list:
        y_batch.append(int(x))

      X_batch = np.array(X_batch)
      y_batch = np.array(y_batch)
      print(f"shape of training batch: {X_batch.shape} \nshape of training target batch: {y_batch.shape}")
      yield X_batch , y_batch

In [ ]:
def build_model():
  resnet_model = Sequential()

  pretrained_model= tf.keras.applications.ResNet101(include_top=False,
                    input_shape=(650 , 650 , 3),
                    pooling='avg',classes=1,
                    weights='imagenet')
  for layer in pretrained_model.layers:
          layer.trainable=False

  resnet_model.add(pretrained_model)
  resnet_model.add(Flatten())
  resnet_model.add(Dense(650, activation='relu'))
  resnet_model.add(Dense(1, activation='sigmoid'))

  return resnet_model

# Model building and Training

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/siddhi2.0/fracture_dataset.csv")
dataset.head()

batch_size = 32  # Number of patients to process per batch
epochs = 4

# Split the labels into training and validation sets
train , val = train_test_split(dataset, test_size=0.2, random_state=42)

steps_per_epoch = len(train) // batch_size
validation_steps = len(val) // batch_size

# Build the model
resnet_model = build_model()

# Train the model using the data generator
train_generator = data_generator(train , batch_size , epochs)
val_generator = data_generator(val, batch_size , epochs)

resnet_model.compile(optimizer=Adam(learning_rate=0.01),loss=binary_crossentropy,metrics=['accuracy'])


171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/siddhi2.0/bone_model/model_epoch_{epoch:02d}.keras',  # Save model with the epoch number in the filename
    monitor='val_loss',      # Monitor validation loss
    save_best_only=True,     # Save only if the model improves
    mode='min',              # Save model with the smallest loss
    verbose=1                              # Print a message when saving
)

In [ ]:
history = resnet_model.fit(
    train_generator,
    steps_per_epoch = steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=[checkpoint_callback]
)

shape of training batch: (32, 650, 650, 3) 
shape of training target batch: (32,)
shape of training batch: (32, 650, 650, 3) 
shape of training target batch: (32,)
Epoch 1/4
  1/111 ━━━━━━━━━━━━━━━━━━━━ 3:11:35 105s/step - accuracy: 0.3750 - loss: 0.7727shape of training batch: (32, 650, 650, 3) 
shape of training target batch: (32,)
  2/111 ━━━━━━━━━━━━━━━━━━━━ 2:37:54 87s/step - accuracy: 0.3672 - loss: 12.5640shape of training batch: (32, 650, 650, 3) 
shape of training target batch: (32,)
  3/111 ━━━━━━━━━━━━━━━━━━━━ 2:38:02 88s/step - accuracy: 0.3906 - loss: 14.0121shape of training batch: (32, 650, 650, 3) 
shape of training target batch: (32,)
  4/111 ━━━━━━━━━━━━━━━━━━━━ 2:35:50 87s/step - accuracy: 0.4121 - loss: 14.5471shape of training batch: (32, 650, 650, 3) 
shape of training target batch: (32,)
  5/111 ━━━━━━━━━━━━━━━━━━━━ 2:34:43 88s/step - accuracy: 0.4234 - loss: 15.0047shape of training batch: (32, 650, 650, 3) 
shape of training target batch: (32,)
  6/111 ━━━━━━━━

In [ ]:
resnet_model.save(r"/content/drive/MyDrive/siddhi2.0/Resnet-101-bone-v3.keras")

In [ ]:
resnet_model.export(r"/content/drive/MyDrive/siddhi2.0/Resnet-101-bone-v3")
# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Temporary


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
import pandas as pd

In [ ]:
import cv2
import numpy as np

In [ ]:
bone_model = load_model("/content/drive/MyDrive/siddhi2.0/bone_model/model_epoch_02.keras")

In [ ]:
frac_Dataset = pd.read_csv('/content/drive/MyDrive/siddhi2.0/fracture_dataset.csv')

In [ ]:
data = frac_Dataset.sample(50)

In [ ]:
data.head()

,Img_path,Fracture_Labels
3102,/content/drive/MyDrive/siddhi2.0/Patients_CT/0...,1
809,/content/drive/MyDrive/siddhi2.0/Patients_CT/0...,0
2650,/content/drive/MyDrive/siddhi2.0/Patients_CT/1...,0
3052,/content/drive/MyDrive/siddhi2.0/Patients_CT/0...,0
1870,/content/drive/MyDrive/siddhi2.0/Patients_CT/0...,1


In [ ]:
path_list = data['Img_path']
y_test = data['Fracture_Labels']

In [ ]:
y_bone_pred = []
for path in path_list:
  img = cv2.imread(path)
  t = np.array([img])
  y = bone_model.predict(t)
  print(y)
  y_bone_pred.extend(y)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.98485154]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.06852463]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.00831056]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.00217028]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.5753863]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[4.841642e-05]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.93365884]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.010132]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.01625879]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.5557324]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.6566651]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.00018184]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.5753863]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.00423805]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.00019544]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[7.91656e-05]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.00297325]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.00991693]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
[[0.21100332]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/ste

KeyboardInterrupt: 

In [ ]:
print(y_test)

3102    1
809     0
2650    0
3052    0
1870    1
3614    0
2586    1
132     0
3205    0
2019    1
1711    1
655     0
282     1
89      0
2558    0
4289    0
658     0
747     0
3141    0
1091    0
3452    1
3011    1
2051    0
1043    1
2623    1
225     1
4290    1
86      0
483     0
4301    0
3486    0
2144    0
1636    0
2714    0
4207    0
3292    1
4141    1
0       1
3937    1
3697    0
4014    0
2361    1
1694    1
2722    1
199     1
1986    0
2943    1
2168    1
1788    0
3958    0
Name: Fracture_Labels, dtype: int64


In [ ]:
model.export("/content/drive/MyDrive/siddhi2.0/bone_model/Res101-boneModel-v3")

Saved artifact at '/content/drive/MyDrive/siddhi2.0/bone_model/Res101-boneModel-v3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 650, 650, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  133395436983168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133395436988096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133395426723616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133395426725552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133395436991616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133395436993904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133395426732768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133395426725904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133395426726784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133395426735232: TensorSpec(shape=()